
## Trabajo Machine Learning

In [1]:
#Librerias

#Libreria del sistema
import os
#Manejo de datos
import pandas as pd
#Separacion de entrenamiento y aprendizaje
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
#Modelo Lasso
from sklearn.linear_model import Lasso
#Metricas del modelo
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score

In [2]:
#Data

#Carga datos
data = pd.read_csv('kc_house_data.csv', sep = ',')

data['date'] = pd.to_datetime(data['date'],format = '%Y-%m-%d')
data.loc[data['yr_renovated'] != 0, 'yr_renovated'] = pd.to_datetime(data.loc[data['yr_renovated'] != 0, 'yr_renovated'], format = '%Y')

df = data

df['yr_renovated_dummy'] = data['yr_renovated'].apply(lambda x: 1 if x!=0 else 0)

df = df.drop(['yr_renovated'], axis=1)

df['age_property'] = 2022 - data['yr_built']


columns = ['price','bedrooms', 'bathrooms', 'sqft_living','waterfront', 'view', 'condition', 'grade', 'yr_renovated_dummy','age_property']

df = df[columns]


In [3]:
#Creacion del modelo usando Lasso

Y = df['price']
X = df.drop(columns = ['price'])

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, random_state = 0)

In [4]:
modelo = Lasso(alpha=1, normalize='True')
modelo.fit(X_train, Y_train)

Lasso(alpha=1, normalize='True')

In [5]:
y_pred = modelo.predict(X_test)

print("MSE: %.2f" % mean_squared_error(Y_test, y_pred, squared =True))
print("MAE: %.2f" % mean_absolute_error(Y_test, y_pred))
print("R^2: %.2f" % r2_score(Y_test, y_pred) )

MSE: 47399111514.82
MAE: 137972.41
R^2: 0.64


In [6]:
particiones = KFold(n_splits=10, shuffle=True, random_state=0)
param_grid = {'alpha': [1e-30,1e-20,1e-15,0.1,0.2,0.3,0.5,0.8,1,1.5,2,3,4,5,10,30]}
modelo = Lasso()

In [7]:
modelo_2 = GridSearchCV(modelo, param_grid, scoring ='neg_mean_squared_error', cv=particiones, n_jobs=-1)
modelo_2.fit(X_train, Y_train)

C:\Users\felip\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 381058779902672.75, tolerance: 219486685028.40982
  model = cd_fast.enet_coordinate_descent(


GridSearchCV(cv=KFold(n_splits=10, random_state=0, shuffle=True),
             estimator=Lasso(), n_jobs=-1,
             param_grid={'alpha': [1e-30, 1e-20, 1e-15, 0.1, 0.2, 0.3, 0.5, 0.8,
                                   1, 1.5, 2, 3, 4, 5, 10, 30]},
             scoring='neg_mean_squared_error')

In [8]:
print("Mejor parámetro: {}".format(modelo_2.best_params_))

Mejor parámetro: {'alpha': 1e-30}


In [9]:
modelo_2 = modelo_2.best_estimator_

y_pred = modelo_2.predict(X_test)

print("R^2: %.2f" % r2_score(Y_test, y_pred) )
print("RMSE: %.2f" % mean_squared_error(Y_test, y_pred, squared = False))
print("MSE: %.2f" % mean_squared_error(Y_test, y_pred, squared =True))
print("MAE: %.2f" % mean_absolute_error(Y_test, y_pred))



R^2: 0.64
RMSE: 217696.90
MSE: 47391938591.92
MAE: 137997.69


In [15]:
#Prueba

bedrooms = 1
bathrooms = 3
sqft_living = 1500
waterfront = 1
view = 1
condition = 3
grade = 7
yr_renovated_dummy = 0
age_property = 12

Precio_modelo = modelo_2.predict([[bedrooms,bathrooms,sqft_living, waterfront,view,condition,grade, yr_renovated_dummy,age_property]])

print("El precio estimadoes: %.2f $$$" % Precio_modelo)

El precio estimadoes: 943816.86 $$$
